## 10. Regular Expression Matching

```
Given an input string (s) and a pattern (p), implement regular expression matching with support for '.' and '*'.

'.' Matches any single character.
'*' Matches zero or more of the preceding element.
The matching should cover the entire input string (not partial).

Note:

s could be empty and contains only lowercase letters a-z.
p could be empty and contains only lowercase letters a-z, and characters like . or *.
Example 1:

Input:
s = "aa"
p = "a"
Output: false
Explanation: "a" does not match the entire string "aa".
Example 2:

Input:
s = "aa"
p = "a*"
Output: true
Explanation: '*' means zero or more of the precedeng element, 'a'. Therefore, by repeating 'a' once, it becomes "aa".
Example 3:

Input:
s = "ab"
p = ".*"
Output: true
Explanation: ".*" means "zero or more (*) of any character (.)".
Example 4:

Input:
s = "aab"
p = "c*a*b"
Output: true
Explanation: c can be repeated 0 times, a can be repeated 1 time. Therefore it matches "aab".
Example 5:

Input:
s = "mississippi"
p = "mis*is*p*."
Output: false
```

重要是解析pattern , p里:
```
a-z
* 
.
```
3*2 6种基本组合

可能由多个pattern构成, 每找到1个 就去字符串里匹配, 如果全部匹配,才true, 否则false

```mis*is*p*.```

In [444]:
# 1 .
# 2 a*
# 3 .*
# 4 a
# 0 * not valid
def is_char(a):
    return True if a in 'abcdefghijklmnopqrstuvwxyz' else False        
def find_pattern(p):
    j = 0
    pattern = 0
    pattern_start = False
    while(j < len(p)):
        
        if not pattern_start:
            if p[j] is '.':  
                pattern = 1
                pattern_start = True
                j+=1
            elif is_char(p[j]):
                pattern = 4
                pattern_start = True
                j+=1
            elif p[j] is '*':
                j+=1
                yield pattern, p[j-1]        
        else:
            if p[j-1] is '.':
                if p[j] is '*':
                    pattern = 3
                    
                    j+=1
                    combine_start = j
                    # 1 .** 
                    # 2 .*a*
                    # .*a*
                    # .*ab
                    # .*abc.*
                    if a[j] is '*' : yield 0,'*'
                    while (j < len(p)):
                        
                        j+=1
                    if p[j-1] is '*': 
                            

                else:
                    yield pattern,p[j-1]
                    
            elif is_char(p[j-1]):
                if p[j] is '*':
                    pattern = 2
                    j+=1
                    yield pattern,p[j-2]
                else:
                    yield pattern,p[j-1]
                    
            pattern_start = False
    if pattern in [1,4]: yield pattern,p[j-1]

        
               
8 

8

In [194]:
[i for i in find_pattern('mis*is*p*.*a')]

[(4, 'm'),
 (4, 'i'),
 (2, 's'),
 (4, 'i'),
 (2, 's'),
 (2, 'p'),
 (3, 'a'),
 (4, 'a')]

In [426]:
[i for i in find_pattern('.*b.....b')]

[(3, 'b'),
 (4, 'b'),
 (1, '.'),
 (1, '.'),
 (1, '.'),
 (1, '.'),
 (1, '.'),
 (4, 'b')]

```sh
.* 处理比较麻烦
.*后 再遇到下1个.* a* 或者结束前,
其余patten 只能 为a or . 
.*[a...a].*
中间的模式需要合并起来,作为一个占位在s里去寻找

重新定义下 pattern ?
```


看下人家的实现,递归,好理解:


In [447]:
class Solution(object):
    def isMatch(self, text, pattern):
        if not pattern:
            return not text

        first_match = bool(text) and pattern[0] in {text[0], '.'}

        if len(pattern) >= 2 and pattern[1] == '*':
            return (self.isMatch(text, pattern[2:]) or
                    first_match and self.isMatch(text[1:], pattern))
        else:
            return first_match and self.isMatch(text[1:], pattern[1:])

```
s = 'abbbbbbbbcd'
p = '.b.*..c.*
```

In [448]:
s = "aa"
p = "aaa"
Solution().isMatch(s,p)

False

In [449]:
s = "ab"
p = ".*c"
Solution().isMatch(s,p)

False

In [450]:
s = "aab"
p = "c*a*b"
assert(Solution().isMatch(s,p))

In [451]:
s = "aa"
p = "a*"
assert(Solution().isMatch(s,p))

In [452]:
s = "ab"
p = ".*"
assert(Solution().isMatch(s,p))

In [453]:
s = "mississippi"
p = "mis*is*p*."
assert(not Solution().isMatch(s,p) )

In [454]:
s = "aa"
p = "a"
assert(not Solution().isMatch(s,p))

In [455]:
s = "asdfsadfsafdsadf"
p = ".*"
assert(Solution().isMatch(s,p))

In [458]:
s = "babb"
p = ".*a.*."
Solution().isMatch(s,p)

True

In [459]:
s = "abbbbb"
p = ".*"
assert(Solution().isMatch(s,p))

In [460]:
s = "abbbbb"
p = ".*bab"
Solution().isMatch(s,p)

False

## 11. Container With Most Water

Given n non-negative integers a1, a2, ..., an , where each represents a point at coordinate (i, ai). n vertical lines are drawn such that the two endpoints of line i is at (i, ai) and (i, 0). Find two lines, which together with x-axis forms a container, such that the container contains the most water.

Note: You may not slant the container and n is at least 2.

```
从第i个开始, 从最远的左端jl往右 or 右端jr往左,找到第1个大于ai的j(j = jr or jl) ,此时最大面积为max= abs(j-i)* ai 

轮询,更新max
```


In [492]:
class Solution:
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        maxs = 0
        ret = None
        for i,ai in enumerate(height):
            left = 0
            right = len(height) -1       
            while(left is not i):
                if height[left] > height[i] :
                    maxer =  height[i] * (i - left)
                    maxs =  maxer if maxer > maxs else maxs 
                    ret = left,i,maxs
                    break
                else:
                    left += 1
            while(right is not i):
                if height[right] > height[i]:
                    maxer =  height[i] * (right - i)
                    maxs =  maxer if maxer > maxs else maxs 
                    ret = right,i,maxs
                    break
                else:
                    right -= 1
        return ret
                
            
            
        

In [493]:
import random

In [495]:
nums = 20
height = [ random.randint(1,100) for i in range(0,20)]
print(height)
Solution().maxArea(height)

[44, 21, 17, 54, 96, 53, 43, 100, 80, 57, 99, 35, 90, 86, 35, 68, 1, 92, 91, 19]


(0, 19, 1274)

In [ ]:
提交超时了...效率确实是个问题.
```

1. 先遍历 end-i 大的那一边; 
```

In [685]:
class Solution:
    def maxArea(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        maxs = 0
        ret = None
        for i,ai in enumerate(height):
            left = 0
            right = len(height) -1       
            while(left is not i):
                if height[left] >= height[i] :
                    maxer =  height[i] * (i - left)
                    maxs =  maxer if maxer > maxs else maxs 
                    ret = left,i,maxs
                    break
                else:
                    left += 1
            while(right is not i):
                if height[right] >= height[i]:
                    maxer =  height[i] * (right - i)
                    maxs =  maxer if maxer > maxs else maxs 
                    ret = right,i,maxs
                    break
                else:
                    right -= 1
        return maxs
    def maxAreaBest(self, height):
        i, j = 0, len(height) - 1
        water = 0
        while i < j:
            water = max(water, (j - i) * min(height[i], height[j]))
            if height[i] < height[j]:
                i += 1
            else:
                j -= 1
        return water
    def maxAreaBetter(self, height):
        """
        :type height: List[int]
        :rtype: int
        """
        maxs = 0 
        left = 0
        right = len(height) -1
        
        for i,ai in enumerate(height):
            def _left(i,left,maxs,dis=0):
                while((i-left ) > dis):
                    if height[left] >= height[i]:
                        maxer =  height[i] * (i - left)
                        maxs =  maxer if maxer > maxs else maxs 
#                         ret = left,i,maxs
                        break
                    else: left += 1
                return i-left,maxs
            
            def _right(i,right,maxs, dis=0):
                while((right -i) > dis):
                    if height[right] >= height[i] :
                        
                        maxer =  height[i] * (right - i)
                        maxs =  maxer if maxer > maxs else maxs 
                        ret = right,i,maxs
#                         print("i = %d , height[right]=%d max = %d "% (i, height[right],maxs))
                        break
                    else: right -= 1
                return right-i,maxs
                
            if (i - left) > (right - i):
                _l,maxs = _left(i,left,maxs,0)
                _,maxs = _right(i,right,maxs,_l)
            else:
                _r,maxs = _right(i,right,maxs,0)
                _,maxs = _left(i,left,maxs,_r)                

        return maxs
                

In [702]:
nums = 20
height = [ random.randint(1,1000) for i in range(0,200000)]
# print(height)
# height=[1,1]
# height = [ i for i in range(0,2000)]
%time ret = Solution().maxArea(height)
print(ret)
%time Solution().maxAreaBetter(height)
print(ret)
%time Solution().maxAreaBest(height)

CPU times: user 495 ms, sys: 0 ns, total: 495 ms
Wall time: 497 ms
198376081
CPU times: user 468 ms, sys: 3.8 ms, total: 472 ms
Wall time: 471 ms
198376081
CPU times: user 126 ms, sys: 0 ns, total: 126 ms
Wall time: 125 ms


198376081

就是顺序list太慢了,针对优化一下?

看看提交的!...被碾压...

```
先从2端开始往中间遍历. 距离肯定是减少的, 求v = 最小高度 * 距离, 只能将希望的小的高度改变,而且要变大. ()

```

In [684]:
class Solution:
    def maxArea(self, height):
        i, j = 0, len(height) - 1
        water = 0
        while i < j:
            water = max(water, (j - i) * min(height[i], height[j]))
            if height[i] < height[j]:
                i += 1
            else:
                j -= 1
        return water

## 15.3Sum

In [705]:
a=[1,2,3]

In [707]:
a[:-1]

[1, 2]

In [886]:
a = set()

In [887]:
a.add(1);a.add(2)

In [888]:
b =set()
b.add(1);b.add(2)
a == b

True

In [970]:
def binary_search(start, end, nums,val):
    mid = (start + end)//2 
    if nums[mid] == val: return mid
    if start >= end: return None
    if nums[mid] > val and mid > start: return binary_search(start,mid-1,nums,val)
    elif nums[mid] < val and mid < end: return binary_search(mid+1,end,nums,val)
    
class Solution:
    def threeSum(self, nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """    
        lists = []
        sets = []
        nums.sort()
        for i in range(0,len(nums)-1):
            for j in range(i+1, len(nums)):    
                pos = None
                pos = binary_search(j+1,len(nums)-1,nums,-(nums[i] + nums[j]))
                if pos  and pos not in [i,j]:
                    items = [nums[i],nums[j],nums[pos]]
                    if items not in lists:
                        lists.append(items)
        return lists
                  

In [971]:
# a = [1,2,3,4,5,7,10,20,30,41]
a = [-14,-3,11,-3,12,-1,11,13,5,6,-11,-14,-6,11,-4,-15,3,-15,9,-10,13,-10,-9,-13,-12,12,-7,12,12,3,9,5,-14,-3,9,13,11,5,3,-6,-12,-1,-5,-3,-4,-2,-10,6,-10,14,3,-11,11,10,-9,7,-1,-9,4,-12,2,-2,8,3,3,-6,-7,-1,6,12,8,9,-12,10,-8,-1,-7,-3,12,-9,-12,1,-5,6,-12,-7,-2,2,-8,-13,5,9,-7,-10,-3,11,-1,-3,-13,-10,-14,11,6,-10,6,13,4,7,-13,-6,13,12,10,-15,4,13,-7,9,-8,0,4,4,-6,12,9,-2,0]
binary_search(0,len(a)-1,a,41)

In [972]:
import random

In [973]:
nums = [random.randint(-20,20) for i in range(0,10)]
# nums = [-14,-3,11,-3,12,-1,11,13,5,6,-11,-14,-6,11,-4,-15,3,-15,9,-10,13,-10,-9,-13,-12,12,-7,12,12,3,9,5,-14,-3,9,13,11,5,3,-6,-12,-1,-5,-3,-4,-2,-10,6,-10,14,3,-11,11,10,-9,7,-1,-9,4,-12,2,-2,8,3,3,-6,-7,-1,6,12,8,9,-12,10,-8,-1,-7,-3,12,-9,-12,1,-5,6,-12,-7,-2,2,-8,-13,5,9,-7,-10,-3,11,-1,-3,-13,-10,-14,11,6,-10,6,13,4,7,-13,-6,13,12,10,-15,4,13,-7,9,-8,0,4,4,-6,12,9,-2,0]

In [974]:
nums = [-1,0,1,2,-1,-4]

In [975]:
%time Solution().threeSum(nums)

CPU times: user 34 µs, sys: 1 µs, total: 35 µs
Wall time: 38.6 µs


[[-1, -1, 2], [-1, 0, 1]]

又超时了??....

In [976]:
nums=[82597,-9243,62390,83030,-97960,-26521,-61011,83390,-38677,12333,75987,46091,83794,19355,-71037,-6242,-28801,324,1202,-90885,-2989,-95597,-34333,35528,5680,89093,-90606,50360,-29393,-27012,53313,65213,99818,-82405,-41661,-3333,-51952,72135,-1523,26377,74685,96992,92263,15929,5467,-99555,-43348,-41689,-60383,-3990,32165,65265,-72973,-58372,12741,-48568,-46596,72419,-1859,34153,62937,81310,-61823,-96770,-54944,8845,-91184,24208,-29078,31495,65258,14198,85395,70506,-40908,56740,-12228,-40072,32429,93001,68445,-73927,25731,-91859,-24150,10093,-60271,-81683,-18126,51055,48189,-6468,25057,81194,-58628,74042,66158,-14452,-49851,-43667,11092,39189,-17025,-79173,13606,83172,92647,-59741,19343,-26644,-57607,82908,-20655,1637,80060,98994,39331,-31274,-61523,91225,-72953,13211,-75116,-98421,-41571,-69074,99587,39345,42151,-2460,98236,15690,-52507,-95803,-48935,-46492,-45606,-79254,-99851,52533,73486,39948,-7240,71815,-585,-96252,90990,-93815,93340,-71848,58733,-14859,-83082,-75794,-82082,-24871,-15206,91207,-56469,-93618,67131,-8682,75719,87429,-98757,-7535,-24890,-94160,85003,33928,75538,97456,-66424,-60074,-8527,-28697,-22308,2246,-70134,-82319,-10184,87081,-34949,-28645,-47352,-83966,-60418,-15293,-53067,-25921,55172,75064,95859,48049,34311,-86931,-38586,33686,-36714,96922,76713,-22165,-80585,-34503,-44516,39217,-28457,47227,-94036,43457,24626,-87359,26898,-70819,30528,-32397,-69486,84912,-1187,-98986,-32958,4280,-79129,-65604,9344,58964,50584,71128,-55480,24986,15086,-62360,-42977,-49482,-77256,-36895,-74818,20,3063,-49426,28152,-97329,6086,86035,-88743,35241,44249,19927,-10660,89404,24179,-26621,-6511,57745,-28750,96340,-97160,-97822,-49979,52307,79462,94273,-24808,77104,9255,-83057,77655,21361,55956,-9096,48599,-40490,-55107,2689,29608,20497,66834,-34678,23553,-81400,-66630,-96321,-34499,-12957,-20564,25610,-4322,-58462,20801,53700,71527,24669,-54534,57879,-3221,33636,3900,97832,-27688,-98715,5992,24520,-55401,-57613,-69926,57377,-77610,20123,52174,860,60429,-91994,-62403,-6218,-90610,-37263,-15052,62069,-96465,44254,89892,-3406,19121,-41842,-87783,-64125,-56120,73904,-22797,-58118,-4866,5356,75318,46119,21276,-19246,-9241,-97425,57333,-15802,93149,25689,-5532,95716,39209,-87672,-29470,-16324,-15331,27632,-39454,56530,-16000,29853,46475,78242,-46602,83192,-73440,-15816,50964,-36601,89758,38375,-40007,-36675,-94030,67576,46811,-64919,45595,76530,40398,35845,41791,67697,-30439,-82944,63115,33447,-36046,-50122,-34789,43003,-78947,-38763,-89210,32756,-20389,-31358,-90526,-81607,88741,86643,98422,47389,-75189,13091,95993,-15501,94260,-25584,-1483,-67261,-70753,25160,89614,-90620,-48542,83889,-12388,-9642,-37043,-67663,28794,-8801,13621,12241,55379,84290,21692,-95906,-85617,-17341,-63767,80183,-4942,-51478,30997,-13658,8838,17452,-82869,-39897,68449,31964,98158,-49489,62283,-62209,-92792,-59342,55146,-38533,20496,62667,62593,36095,-12470,5453,-50451,74716,-17902,3302,-16760,-71642,-34819,96459,-72860,21638,47342,-69897,-40180,44466,76496,84659,13848,-91600,-90887,-63742,-2156,-84981,-99280,94326,-33854,92029,-50811,98711,-36459,-75555,79110,-88164,-97397,-84217,97457,64387,30513,-53190,-83215,252,2344,-27177,-92945,-89010,82662,-11670,86069,53417,42702,97082,3695,-14530,-46334,17910,77999,28009,-12374,15498,-46941,97088,-35030,95040,92095,-59469,-24761,46491,67357,-66658,37446,-65130,-50416,99197,30925,27308,54122,-44719,12582,-99525,-38446,-69050,-22352,94757,-56062,33684,-40199,-46399,96842,-50881,-22380,-65021,40582,53623,-76034,77018,-97074,-84838,-22953,-74205,79715,-33920,-35794,-91369,73421,-82492,63680,-14915,-33295,37145,76852,-69442,60125,-74166,74308,-1900,-30195,-16267,-60781,-27760,5852,38917,25742,-3765,49097,-63541,98612,-92865,-30248,9612,-8798,53262,95781,-42278,-36529,7252,-27394,-5021,59178,80934,-48480,-75131,-54439,-19145,-48140,98457,-6601,-51616,-89730,78028,32083,-48904,16822,-81153,-8832,48720,-80728,-45133,-86647,-4259,-40453,2590,28613,50523,-4105,-27790,-74579,-17223,63721,33489,-47921,97628,-97691,-14782,-65644,18008,-93651,-71266,80990,-76732,-47104,35368,28632,59818,-86269,-89753,34557,-92230,-5933,-3487,-73557,-13174,-43981,-43630,-55171,30254,-83710,-99583,-13500,71787,5017,-25117,-78586,86941,-3251,-23867,-36315,75973,86272,-45575,77462,-98836,-10859,70168,-32971,-38739,-12761,93410,14014,-30706,-77356,-85965,-62316,63918,-59914,-64088,1591,-10957,38004,15129,-83602,-51791,34381,-89382,-26056,8942,5465,71458,-73805,-87445,-19921,-80784,69150,-34168,28301,-68955,18041,6059,82342,9947,39795,44047,-57313,48569,81936,-2863,-80932,32976,-86454,-84207,33033,32867,9104,-16580,-25727,80157,-70169,53741,86522,84651,68480,84018,61932,7332,-61322,-69663,76370,41206,12326,-34689,17016,82975,-23386,39417,72793,44774,-96259,3213,79952,29265,-61492,-49337,14162,65886,3342,-41622,-62659,-90402,-24751,88511,54739,-21383,-40161,-96610,-24944,-602,-76842,-21856,69964,43994,-15121,-85530,12718,13170,-13547,69222,62417,-75305,-81446,-38786,-52075,-23110,97681,-82800,-53178,11474,35857,94197,-58148,-23689,32506,92154,-64536,-73930,-77138,97446,-83459,70963,22452,68472,-3728,-25059,-49405,95129,-6167,12808,99918,30113,-12641,-26665,86362,-33505,50661,26714,33701,89012,-91540,40517,-12716,-57185,-87230,29914,-59560,13200,-72723,58272,23913,-45586,-96593,-26265,-2141,31087,81399,92511,-34049,20577,2803,26003,8940,42117,40887,-82715,38269,40969,-50022,72088,21291,-67280,-16523,90535,18669,94342,-39568,-88080,-99486,-20716,23108,-28037,63342,36863,-29420,-44016,75135,73415,16059,-4899,86893,43136,-7041,33483,-67612,25327,40830,6184,61805,4247,81119,-22854,-26104,-63466,63093,-63685,60369,51023,51644,-16350,74438,-83514,99083,10079,-58451,-79621,48471,67131,-86940,99093,11855,-22272,-67683,-44371,9541,18123,37766,-70922,80385,-57513,-76021,-47890,36154,72935,84387,-92681,-88303,-7810,59902,-90,-64704,-28396,-66403,8860,13343,33882,85680,7228,28160,-14003,54369,-58893,92606,-63492,-10101,64714,58486,29948,-44679,-22763,10151,-56695,4031,-18242,-36232,86168,-14263,9883,47124,47271,92761,-24958,-73263,-79661,-69147,-18874,29546,-92588,-85771,26451,-86650,-43306,-59094,-47492,-34821,-91763,-47670,33537,22843,67417,-759,92159,63075,94065,-26988,55276,65903,30414,-67129,-99508,-83092,-91493,-50426,14349,-83216,-76090,32742,-5306,-93310,-60750,-60620,-45484,-21108,-58341,-28048,-52803,69735,78906,81649,32565,-86804,-83202,-65688,-1760,89707,93322,-72750,84134,71900,-37720,19450,-78018,22001,-23604,26276,-21498,65892,-72117,-89834,-23867,55817,-77963,42518,93123,-83916,63260,-2243,-97108,85442,-36775,17984,-58810,99664,-19082,93075,-69329,87061,79713,16296,70996,13483,-74582,49900,-27669,-40562,1209,-20572,34660,83193,75579,7344,64925,88361,60969,3114,44611,-27445,53049,-16085,-92851,-53306,13859,-33532,86622,-75666,-18159,-98256,51875,-42251,-27977,-18080,23772,38160,41779,9147,94175,99905,-85755,62535,-88412,-52038,-68171,93255,-44684,-11242,-104,31796,62346,-54931,-55790,-70032,46221,56541,-91947,90592,93503,4071,20646,4856,-63598,15396,-50708,32138,-85164,38528,-89959,53852,57915,-42421,-88916,-75072,67030,-29066,49542,-71591,61708,-53985,-43051,28483,46991,-83216,80991,-46254,-48716,39356,-8270,-47763,-34410,874,-1186,-7049,28846,11276,21960,-13304,-11433,-4913,55754,79616,70423,-27523,64803,49277,14906,-97401,-92390,91075,70736,21971,-3303,55333,-93996,76538,54603,-75899,98801,46887,35041,48302,-52318,55439,24574,14079,-24889,83440,14961,34312,-89260,-22293,-81271,-2586,-71059,-10640,-93095,-5453,-70041,66543,74012,-11662,-52477,-37597,-70919,92971,-17452,-67306,-80418,7225,-89296,24296,86547,37154,-10696,74436,-63959,58860,33590,-88925,-97814,-83664,85484,-8385,-50879,57729,-74728,-87852,-15524,-91120,22062,28134,80917,32026,49707,-54252,-44319,-35139,13777,44660,85274,25043,58781,-89035,-76274,6364,-63625,72855,43242,-35033,12820,-27460,77372,-47578,-61162,-70758,-1343,-4159,64935,56024,-2151,43770,19758,-30186,-86040,24666,-62332,-67542,73180,-25821,-27826,-45504,-36858,-12041,20017,-24066,-56625,-52097,-47239,-90694,8959,7712,-14258,-5860,55349,61808,-4423,-93703,64681,-98641,-25222,46999,-83831,-54714,19997,-68477,66073,51801,-66491,52061,-52866,79907,-39736,-68331,68937,91464,98892,910,93501,31295,-85873,27036,-57340,50412,21,-2445,29471,71317,82093,-94823,-54458,-97410,39560,-7628,66452,39701,54029,37906,46773,58296,60370,-61090,85501,-86874,71443,-72702,-72047,14848,34102,77975,-66294,-36576,31349,52493,-70833,-80287,94435,39745,-98291,84524,-18942,10236,93448,50846,94023,-6939,47999,14740,30165,81048,84935,-19177,-13594,32289,62628,-90612,-542,-66627,64255,71199,-83841,-82943,-73885,8623,-67214,-9474,-35249,62254,-14087,-90969,21515,-83303,94377,-91619,19956,-98810,96727,-91939,29119,-85473,-82153,-69008,44850,74299,-76459,-86464,8315,-49912,-28665,59052,-69708,76024,-92738,50098,18683,-91438,18096,-19335,35659,91826,15779,-73070,67873,-12458,-71440,-46721,54856,97212,-81875,35805,36952,68498,81627,-34231,81712,27100,-9741,-82612,18766,-36392,2759,41728,69743,26825,48355,-17790,17165,56558,3295,-24375,55669,-16109,24079,73414,48990,-11931,-78214,90745,19878,35673,-15317,-89086,94675,-92513,88410,-93248,-19475,-74041,-19165,32329,-26266,-46828,-18747,45328,8990,-78219,-25874,-74801,-44956,-54577,-29756,-99822,-35731,-18348,-68915,-83518,-53451,95471,-2954,-13706,-8763,-21642,-37210,16814,-60070,-42743,27697,-36333,-42362,11576,85742,-82536,68767,-56103,-63012,71396,-78464,-68101,-15917,-11113,-3596,77626,-60191,-30585,-73584,6214,-84303,18403,23618,-15619,-89755,-59515,-59103,-74308,-63725,-29364,-52376,-96130,70894,-12609,50845,-2314,42264,-70825,64481,55752,4460,-68603,-88701,4713,-50441,-51333,-77907,97412,-66616,-49430,60489,-85262,-97621,-18980,44727,-69321,-57730,66287,-92566,-64427,-14270,11515,-92612,-87645,61557,24197,-81923,-39831,-10301,-23640,-76219,-68025,92761,-76493,68554,-77734,-95620,-11753,-51700,98234,-68544,-61838,29467,46603,-18221,-35441,74537,40327,-58293,75755,-57301,-7532,-94163,18179,-14388,-22258,-46417,-48285,18242,-77551,82620,250,-20060,-79568,-77259,82052,-98897,-75464,48773,-79040,-11293,45941,-67876,-69204,-46477,-46107,792,60546,-34573,-12879,-94562,20356,-48004,-62429,96242,40594,2099,99494,25724,-39394,-2388,-18563,-56510,-83570,-29214,3015,74454,74197,76678,-46597,60630,-76093,37578,-82045,-24077,62082,-87787,-74936,58687,12200,-98952,70155,-77370,21710,-84625,-60556,-84128,925,65474,-15741,-94619,88377,89334,44749,22002,-45750,-93081,-14600,-83447,46691,85040,-66447,-80085,56308,44310,24979,-29694,57991,4675,-71273,-44508,13615,-54710,23552,-78253,-34637,50497,68706,81543,-88408,-21405,6001,-33834,-21570,-46692,-25344,20310,71258,-97680,11721,59977,59247,-48949,98955,-50276,-80844,-27935,-76102,55858,-33492,40680,66691,-33188,8284,64893,-7528,6019,-85523,8434,-64366,-56663,26862,30008,-7611,-12179,-70076,21426,-11261,-36864,-61937,-59677,929,-21052,3848,-20888,-16065,98995,-32293,-86121,-54564,77831,68602,74977,31658,40699,29755,98424,80358,-69337,26339,13213,-46016,-18331,64713,-46883,-58451,-70024,-92393,-4088,70628,-51185,71164,-75791,-1636,-29102,-16929,-87650,-84589,-24229,-42137,-15653,94825,13042,88499,-47100,-90358,-7180,29754,-65727,-42659,-85560,-9037,-52459,20997,-47425,17318,21122,20472,-23037,65216,-63625,-7877,-91907,24100,-72516,22903,-85247,-8938,73878,54953,87480,-31466,-99524,35369,-78376,89984,-15982,94045,-7269,23319,-80456,-37653,-76756,2909,81936,54958,-12393,60560,-84664,-82413,66941,-26573,-97532,64460,18593,-85789,-38820,-92575,-43663,-89435,83272,-50585,13616,-71541,-53156,727,-27644,16538,34049,57745,34348,35009,16634,-18791,23271,-63844,95817,21781,16590,59669,15966,-6864,48050,-36143,97427,-59390,96931,78939,-1958,50777,43338,-51149,39235,-27054,-43492,67457,-83616,37179,10390,85818,2391,73635,87579,-49127,-81264,-79023,-81590,53554,-74972,-83940,-13726,-39095,29174,78072,76104,47778,25797,-29515,-6493,-92793,22481,-36197,-65560,42342,15750,97556,99634,-56048,-35688,13501,63969,-74291,50911,39225,93702,-3490,-59461,-30105,-46761,-80113,92906,-68487,50742,36152,-90240,-83631,24597,-50566,-15477,18470,77038,40223,-80364,-98676,70957,-63647,99537,13041,31679,86631,37633,-16866,13686,-71565,21652,-46053,-80578,-61382,68487,-6417,4656,20811,67013,-30868,-11219,46,74944,14627,56965,42275,-52480,52162,-84883,-52579,-90331,92792,42184,-73422,-58440,65308,-25069,5475,-57996,59557,-17561,2826,-56939,14996,-94855,-53707,99159,43645,-67719,-1331,21412,41704,31612,32622,1919,-69333,-69828,22422,-78842,57896,-17363,27979,-76897,35008,46482,-75289,65799,20057,7170,41326,-76069,90840,-81253,-50749,3649,-42315,45238,-33924,62101,96906,58884,-7617,-28689,-66578,62458,50876,-57553,6739,41014,-64040,-34916,37940,13048,-97478,-11318,-89440,-31933,-40357,-59737,-76718,-14104,-31774,28001,4103,41702,-25120,-31654,63085,-3642,84870,-83896,-76422,-61520,12900,88678,85547,33132,-88627,52820,63915,-27472,78867,-51439,33005,-23447,-3271,-39308,39726,-74260,-31874,-36893,93656,910,-98362,60450,-88048,99308,13947,83996,-90415,-35117,70858,-55332,-31721,97528,82982,-86218,6822,25227,36946,97077,-4257,-41526,56795,89870,75860,-70802,21779,14184,-16511,-89156,-31422,71470,69600,-78498,74079,-19410,40311,28501,26397,-67574,-32518,68510,38615,19355,-6088,-97159,-29255,-92523,3023,-42536,-88681,64255,41206,44119,52208,39522,-52108,91276,-70514,83436,63289,-79741,9623,99559,12642,85950,83735,-21156,-67208,98088,-7341,-27763,-30048,-44099,-14866,-45504,-91704,19369,13700,10481,-49344,-85686,33994,19672,36028,60842,66564,-24919,33950,-93616,-47430,-35391,-28279,56806,74690,39284,-96683,-7642,-75232,37657,-14531,-86870,-9274,-26173,98640,88652,64257,46457,37814,-19370,9337,-22556,-41525,39105,-28719,51611,-93252,98044,-90996,21710,-47605,-64259,-32727,53611,-31918,-3555,33316,-66472,21274,-37731,-2919,15016,48779,-88868,1897,41728,46344,-89667,37848,68092,-44011,85354,-43776,38739,-31423,-66330,65167,-22016,59405,34328,-60042,87660,-67698,-59174,-1408,-46809,-43485,-88807,-60489,13974,22319,55836,-62995,-37375,-4185,32687,-36551,-75237,58280,26942,-73756,71756,78775,-40573,14367,-71622,-77338,24112,23414,-7679,-51721,87492,85066,-21612,57045,10673,-96836,52461,-62218,-9310,65862,-22748,89906,-96987,-98698,26956,-43428,46141,47456,28095,55952,67323,-36455,-60202,-43302,-82932,42020,77036,10142,60406,70331,63836,58850,-66752,52109,21395,-10238,-98647,-41962,27778,69060,98535,-28680,-52263,-56679,66103,-42426,27203,80021,10153,58678,36398,63112,34911,20515,62082,-15659,-40785,27054,43767,-20289,65838,-6954,-60228,-72226,52236,-35464,25209,-15462,-79617,-41668,-84083,62404,-69062,18913,46545,20757,13805,24717,-18461,-47009,-25779,68834,64824,34473,39576,31570,14861,-15114,-41233,95509,68232,67846,84902,-83060,17642,-18422,73688,77671,-26930,64484,-99637,73875,6428,21034,-73471,19664,-68031,15922,-27028,48137,54955,-82793,-41144,-10218,-24921,-28299,-2288,68518,-54452,15686,-41814,66165,-72207,-61986,80020,50544,-99500,16244,78998,40989,14525,-56061,-24692,-94790,21111,37296,-90794,72100,70550,-31757,17708,-74290,61910,78039,-78629,-25033,73172,-91953,10052,64502,99585,-1741,90324,-73723,68942,28149,30218,24422,16659,10710,-62594,94249,96588,46192,34251,73500,-65995,-81168,41412,-98724,-63710,-54696,-52407,19746,45869,27821,-94866,-76705,-13417,-61995,-71560,43450,67384,-8838,-80293,-28937,23330,-89694,-40586,46918,80429,-5475,78013,25309,-34162,37236,-77577,86744,26281,-29033,-91813,35347,13033,-13631,-24459,3325,-71078,-75359,81311,19700,47678,-74680,-84113,45192,35502,37675,19553,76522,-51098,-18211,89717,4508,-82946,27749,85995,89912,-53678,-64727,-14778,32075,-63412,-40524,86440,-2707,-36821,63850,-30883,67294,-99468,-23708,34932,34386,98899,29239,-23385,5897,54882,98660,49098,70275,17718,88533,52161,63340,50061,-89457,19491,-99156,24873,-17008,64610,-55543,50495,17056,-10400,-56678,-29073,-42960,-76418,98562,-88104,-96255,10159,-90724,54011,12052,45871,-90933,-69420,67039,37202,78051,-52197,-40278,-58425,65414,-23394,-1415,6912,-53447,7352,17307,-78147,63727,98905,55412,-57658,-32884,-44878,22755,39730,3638,35111,39777,74193,38736,-11829,-61188,-92757,55946,-71232,-63032,-83947,39147,-96684,-99233,25131,-32197,24406,-55428,-61941,25874,-69453,64483,-19644,-68441,12783,87338,-48676,66451,-447,-61590,50932,-11270,29035,65698,-63544,10029,80499,-9461,86368,91365,-81810,-71914,-52056,-13782,44240,-30093,-2437,24007,67581,-17365,-69164,-8420,-69289,-29370,48010,90439,13141,69243,50668,39328,61731,78266,-81313,17921,-38196,55261,9948,-24970,75712,-72106,28696,7461,31621,61047,51476,56512,11839,-96916,-82739,28924,-99927,58449,37280,69357,11219,-32119,-62050,-48745,-83486,-52376,42668,82659,68882,38773,46269,-96005,97630,25009,-2951,-67811,99801,81587,-79793,-18547,-83086,69512,33127,-92145,-88497,47703,59527,1909,88785,-88882,69188,-46131,-5589,-15086,36255,-53238,-33009,82664,53901,35939,-42946,-25571,33298,69291,53199,74746,-40127,-39050,91033,51717,-98048,87240,36172,65453,-94425,-63694,-30027,59004,88660,3649,-20267,-52565,-67321,34037,4320,91515,-56753,60115,27134,68617,-61395,-26503,-98929,-8849,-63318,10709,-16151,61905,-95785,5262,23670,-25277,90206,-19391,45735,37208,-31992,-92450,18516,-90452,-58870,-58602,93383,14333,17994,82411,-54126,-32576,35440,-60526,-78764,-25069,-9022,-394,92186,-38057,55328,-61569,67780,77169,19546,-92664,-94948,44484,-13439,83529,27518,-48333,72998,38342,-90553,-98578,-76906,81515,-16464,78439,92529,35225,-39968,-10130,-7845,-32245,-74955,-74996,67731,-13897,-82493,33407,93619,59560,-24404,-57553,19486,-45341,34098,-24978,-33612,79058,71847,76713,-95422,6421,-96075,-59130,-28976,-16922,-62203,69970,68331,21874,40551,89650,51908,58181,66480,-68177,34323,-3046,-49656,-59758,43564,-10960,-30796,15473,-20216,46085,-85355,41515,-30669,-87498,57711,56067,63199,-83805,62042,91213,-14606,4394,-562,74913,10406,96810,-61595,32564,31640,-9732,42058,98052,-7908,-72330,1558,-80301,34878,32900,3939,-8824,88316,20937,21566,-3218,-66080,-31620,86859,54289,90476,-42889,-15016,-18838,75456,30159,-67101,42328,-92703,85850,-5475,23470,-80806,68206,17764,88235,46421,-41578,74005,-81142,80545,20868,-1560,64017,83784,68863,-97516,-13016,-72223,79630,-55692,82255,88467,28007,-34686,-69049,-41677,88535,-8217,68060,-51280,28971,49088,49235,26905,-81117,-44888,40623,74337,-24662,97476,79542,-72082,-35093,98175,-61761,-68169,59697,-62542,-72965,59883,-64026,-37656,-92392,-12113,-73495,98258,68379,-21545,64607,-70957,-92254,-97460,-63436,-8853,-19357,-51965,-76582,12687,-49712,45413,-60043,33496,31539,-57347,41837,67280,-68813,52088,-13155,-86430,-15239,-45030,96041,18749,-23992,46048,35243,-79450,85425,-58524,88781,-39454,53073,-48864,-82289,39086,82540,-11555,25014,-5431,-39585,-89526,2705,31953,-81611,36985,-56022,68684,-27101,11422,64655,-26965,-63081,-13840,-91003,-78147,-8966,41488,1988,99021,-61575,-47060,65260,-23844,-21781,-91865,-19607,44808,2890,63692,-88663,-58272,15970,-65195,-45416,-48444,-78226,-65332,-24568,42833,-1806,-71595,80002,-52250,30952,48452,-90106,31015,-22073,62339,63318,78391,28699,77900,-4026,-76870,-45943,33665,9174,-84360,-22684,-16832,-67949,-38077,-38987,-32847,51443,-53580,-13505,9344,-92337,26585,70458,-52764,-67471,-68411,-1119,-2072,-93476,67981,40887,-89304,-12235,41488,1454,5355,-34855,-72080,24514,-58305,3340,34331,8731,77451,-64983,-57876,82874,62481,-32754,-39902,22451,-79095,-23904,78409,-7418,77916]

N^2*logN 系统都嫌太慢?,看下别人的答案...

In [978]:
from collections import Counter
class Solution:
    def threeSum(self, nums):
        nums.sort()
        d=Counter(nums)
        a, b =set(), set()
        for k in nums:
            d[k]-=1
            for i in a:
                if  d[-(k+i)]>0 :
                    b.add((k,i,-(k+i)))
            a.add(k)
        return list(b) 

```
Counter  dict中: 10 个0  9个8 8个-8 ...
d[0] = 10, d[8] = 9 , d[-8] = 8
遇到0, d[0] -=1 , d[0] = 9, 
0 放到set a中
遇到 8 d[8] -= 1 d[8] = 7
d[-8=-(0+8)] != 0? ,那么(8,0,-8)就是要找的元素对, 

```

## 19. Remove Nth Node From End of List

```
Given linked list: 1->2->3->4->5, and n = 2.

After removing the second node from the end, the linked list becomes 1->2->3->5.
```

思路1  链表逆向 然后顺着 move

思路2  先统计链表长度,然后计算顺的位置

In [727]:
def dumpList(p):
    while(p.next != None):
        print('%d '%(p.val))
        p = p.next
    print('%d '%(p.val))
    
def getListLen(head):
    l = 0
    if head == None: return 0
    p = head
    while(p.next != None):
        p = p.next
        l +=1
    return l+1

class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def removeNthFromEnd(self, head, n):
        """
        :type head: ListNode
        :type n: int
        :rtype: ListNode
        """
        lens = getListLen(head)
        dumpList(head)
        print('lens=',lens)
        # 5 - 2 = 3 0,1,2,3
        pos = lens - n
        p = head
        if pos == 0: 
            head = head.next
            return head
        prev = None
        while(pos) : 
            prev = p
            p = p.next
            pos -=1
        prev.next = p.next
        return head  

In [728]:
head  = p  = ListNode(1)
for i in range(2,5+2):
    p.next =  ListNode(i)
    p = p.next
dumpList(head)

print('-----')
    
head = Solution().removeNthFromEnd(head,1)
dumpList(head)

1 
2 
3 
4 
5 
6 
-----
1 
2 
3 
4 
5 
6 
lens= 6
1 
2 
3 
4 
5 


## 61.  Rotate List

```
Input: 1->2->3->4->5->NULL, k = 2
Output: 4->5->1->2->3->NULL
Explanation:
rotate 1 steps to the right: 5->1->2->3->4->NULL
rotate 2 steps to the right: 4->5->1->2->3->NULL
```

```
Input: 0->1->2->NULL, k = 4
Output: 2->0->1->NULL
Explanation:
rotate 1 steps to the right: 2->0->1->NULL
rotate 2 steps to the right: 1->2->0->NULL
rotate 3 steps to the right: 0->1->2->NULL
rotate 4 steps to the right: 2->0->1->NULL
```

In [ ]:
思路 1  每次迭代时, 把tail remove出来, 保存, 然后 insert_before head

In [740]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None
        
def getListLen(head):
    l = 0
    if head == None: return 0
    p = head
    while(p.next != None):
        p = p.next
        l +=1
    return l+1

class Solution:
    def rotateRight(self, head, k):
        """
        :type head: ListNode
        :type k: int
        :rtype: ListNode
        """
        l = getListLen(head)
        p = head 
        if p == None: return None
        if p.next ==None: return head
        prev = None
        for it in range(k % l):
            while( p.next != None):
                prev =p
                p = p.next
            prev.next = p.next
            
            # remove item as new head
            p.next = head
            head = p
        return head
        

In [746]:
head  = p  = ListNode(1)
for i in range(2,5+2):
    p.next =  ListNode(i)
    p = p.next
dumpList(head)

print('-----')
    
head = Solution().rotateRight(head,2)
dumpList(head)

1 
2 
3 
4 
5 
6 
-----
5 
6 
1 
2 
3 
4 


思路没毛病!!但是 超时了... 没必要每个k都这么算, k % l的结果是相同的..

提交后还是很慢, 再自己看 可以整体搬移的 ! 没必要1个1个的item挪

## 83. Remove Duplicates from Sorted List

```
Input: 1->1->2
Output: 1->2
```

In [797]:
def dumpList(p):
    while(p.next != None):
        print('%d '%(p.val))
        p = p.next
    print('%d '%(p.val))
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def deleteDuplicates(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        p=head
        nexts=None
        if p.next ==None: return head
        while(p.next != None ):
            
            while p.val == p.next.val:
                nexts= p.next
                if nexts == None :
                    break
                if nexts.next != None: 
                    p.next = nexts.next
                    nexts = nexts.next
                else:
                    p.next =None
                    break
                    
            if p.next != None:
                p = p.next
            else:
                break
            
        return head
        

In [800]:
head  = p  = ListNode(1)
p.next =  ListNode(1)
p = p.next
p.next =  ListNode(2)
p = p.next

p.next =  ListNode(2)
p = p.next

p.next =  ListNode(3)
p = p.next
p.next =  ListNode(3)

dumpList(head)

1 
1 
2 
2 
3 
3 


In [801]:
Solution().deleteDuplicates(head)
dumpList(head)

1 
2 
3 


In [983]:
a=[1,0,0,1]
a[0],a[3] = a[3],a[0]
a[1],a[2] = a[2],a[1]
a


[1, 0, 0, 1]

# 按array, string, linklist , tree, math, 从easy 到medium 到hard 来刷 

## flipAndInvertImage

In [990]:
class Solution:
    def flipAndInvertImage(self, A):
        """
        :type A: List[List[int]]
        :rtype: List[List[int]]
        """
        lens = len(A[0])
        mid = lens // 2
        for l in A:
            end = lens
            m = 0
            if lens & 1:
                while(m < mid):
                    l[m],l[end-m-1] = l[end-m-1],l[m]
                    l[m]^=1
                    l[end-m-1]^=1
                    m+=1
                l[mid]^=1
            else:
                while(m < mid):
                    l[m],l[end-m-1] = l[end-m-1],l[m]
                    l[m]^=1
                    l[end-m-1]^=1
                    m+=1
        return A
                    
                

In [993]:
a = [0, 1, 0, 1]
a[0],a[3]=a[3],a[0]
a[0]^=1
a[3]^=1

[1, 0, 1, 0]

In [991]:
Solution().flipAndInvertImage([[1,1,0,0],[1,0,0,1],[0,1,1,1],[1,0,1,0]])

[1, 1, 0, 0]
0 [0, 1, 0, 1]
flip 0 [1, 1, 0, 0]
1 [1, 0, 1, 0]
flip 1 [1, 1, 0, 0]
2 [1, 0, 1, 0]
flip 2 [1, 1, 0, 0]
[1, 0, 0, 1]
0 [1, 0, 0, 1]
flip 0 [0, 0, 0, 0]
1 [0, 0, 0, 0]
flip 1 [0, 1, 1, 0]
2 [0, 1, 1, 0]
flip 2 [0, 0, 0, 0]
[0, 1, 1, 1]
0 [1, 1, 1, 0]
flip 0 [0, 1, 1, 1]
1 [0, 1, 1, 1]
flip 1 [0, 0, 0, 1]
2 [0, 0, 0, 1]
flip 2 [0, 1, 1, 1]
[1, 0, 1, 0]
0 [0, 0, 1, 1]
flip 0 [1, 0, 1, 0]
1 [1, 1, 0, 0]
flip 1 [1, 0, 1, 0]
2 [1, 1, 0, 0]
flip 2 [1, 0, 1, 0]


[[1, 1, 0, 0], [0, 0, 0, 0], [0, 1, 1, 1], [1, 0, 1, 0]]

## 561. Array Partition I

```
Given an array of 2n integers, your task is to group these integers into n pairs of integer, say (a1, b1), (a2, b2), ..., (an, bn) which makes sum of min(ai, bi) for all i from 1 to n as large as possible.

Example 1:
Input: [1,4,3,2]

Output: 4
Explanation: n is 2, and the maximum sum of pairs is 4 = min(1, 2) + min(3, 4).
Note:
n is a positive integer, which is in the range of [1, 10000].
All the integers in the array will be in the range of [-10000, 10000].
```

最小值 希望大一点, 每个pair,尽量把最小的放在前面的pair里,后面的pair的最小值会越来越大.


In [995]:
sum([1,2])

3

In [996]:
a=[1,2,3,4]
a[::2]

[1, 3]

In [997]:
import random

In [1009]:
a = [random.randint(-1000,1000) for i in range(0,10)]

In [1010]:
a[::]

[904, 44, -264, 40, 110, 282, 761, 658, 303, -966]

In [1011]:
a=[1,2,3]


In [1012]:
a.insert(0,4)

In [1013]:
a

[4, 1, 2, 3]

In [1014]:
class Solution:
    def sortArrayByParityII(self, A):
        """
        :type A: List[int]
        :rtype: List[int]
        """
        B=A[:]
        even=0
        odd=1
        for i in range(0,len(A)):
            if A[i] & 1: 
                B[odd] = A[i]
                odd+=2
            else:
                B[even] = A[i]
                even+=2
        return B
            

## 867. Transpose Matrix

```
Given a matrix A, return the transpose of A.

The transpose of a matrix is the matrix flipped over it's main diagonal, switching the row and column indices of the matrix.

 

Example 1:

Input: [[1,2,3],[4,5,6],[7,8,9]]
Output: [[1,4,7],[2,5,8],[3,6,9]]
Example 2:

Input: [[1,2,3],[4,5,6]]
Output: [[1,4],[2,5],[3,6]]
```

In [1017]:
a=[[1,1],[2,2]]
a[1][0]

2

In [1020]:
class Solution:
    def transpose(self, A):
        """
        :type A: List[List[int]]
        :rtype: List[List[int]]
        """
        row = len(A)
        col = len(A[0])
        
        B=[]
        
        for i in range(0,col):
            B.append([])
            for j in range(0,row):
                B[i].append(A[j][i]) 
        return B
        

In [1022]:
Solution().transpose([[1,2,3],[4,5,6]])

[[1, 4], [2, 5], [3, 6]]

略慢,append?

In [1103]:
a=[[1,1],[2,2]]

SyntaxError: can't use starred expression here (cell_name, line 5)

In [1104]:
[*zip([1,1],[2,2],[3,3])]

[(1, 2, 3), (1, 2, 3)]

In [ ]:
class Solution:
    def transpose(self, A):
        """
        :type A: List[List[int]]
        :rtype: List[List[int]]
        """
        return [*zip(*A)]
       # return [[row[i] for row in A] for i in range(len(A[0]))]	

## 766. Toeplitz Matrix


```
A matrix is Toeplitz if every diagonal from top-left to bottom-right has the same element.

Now given an M x N matrix, return True if and only if the matrix is Toeplitz.
 

Example 1:

Input:
matrix = [
  [1,2,3,4],
  [5,1,2,3],
  [9,5,1,2]
]
Output: True
Explanation:
In the above grid, the diagonals are:
"[9]", "[5, 5]", "[1, 1, 1]", "[2, 2, 2]", "[3, 3]", "[4]".
In each diagonal all elements are the same, so the answer is True.
```

In [1026]:
a=[[1,1],[2,2],[3,3]]
a[-1]

[3, 3]

In [1096]:
class Solution:
    def isToeplitzMatrix(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: bool
        """
        for i in range(len(matrix) - 1):
            for j in range(len(matrix[0]) - 1):
                if matrix[i][j] != matrix[i + 1][j + 1]:
                    return False
        return True
            
            
        

        

In [1097]:
Solution().isToeplitzMatrix([[1,2,3,4],[5,1,2,3],[9,5,1,2]])

True

In [1098]:
Solution().isToeplitzMatrix([[1,2],[2,2]])

False

In [1099]:

Solution().isToeplitzMatrix([[1,2,3,4],[5,1,2,3],[9,5,1,2]]) 

True

## 566. Reshape the Matrix

```
In MATLAB, there is a very useful function called 'reshape', which can reshape a matrix into a new one with different size but keep its original data.

You're given a matrix represented by a two-dimensional array, and two positive integers r and c representing the row number and column number of the wanted reshaped matrix, respectively.

The reshaped matrix need to be filled with all the elements of the original matrix in the same row-traversing order as they were.

If the 'reshape' operation with given parameters is possible and legal, output the new reshaped matrix; Otherwise, output the original matrix.

Example 1:
Input: 
nums = 
[[1,2],
 [3,4]]
r = 1, c = 4
Output: 
[[1,2,3,4]]
Explanation:
The row-traversing of nums is [1,2,3,4]. The new reshaped matrix is a 1 * 4 matrix, fill it row by row by using the previous list.
Example 2:
Input: 
nums = 
[[1,2],
 [3,4]]
r = 2, c = 4
Output: 
[[1,2],
 [3,4]]
Explanation:
There is no way to reshape a 2 * 2 matrix to a 2 * 4 matrix. So output the original matrix.
Note:
The height and width of the given matrix is in range [1, 100].
The given r and c are all positive.
```

In [1109]:
a=[1,2,3,4]
c= 2
print(a[0*c:1*c])
print(a[1*c:2*c])

[1, 2]
[3, 4]


In [1138]:
class Solution:
    def matrixReshape(self, nums, r, c):
        """
        :type nums: List[List[int]]
        :type r: int
        :type c: int
        :rtype: List[List[int]]
        """
        _r = len(nums)
        _c = len(nums[0])
        
        print(_r,_c,r,c)
        if _r *_c != r * c : return nums
        
        
        A =[]
        for n in nums:
            A += n
        B=[]
        for i in range(0,r):
            B.append([])
            B[i] = A[i*c:(i+1)*c]
        return B
            

In [1140]:
len(m)

76

In [1141]:
67*76 

5092

In [1142]:
134*38

5092

In [1143]:
m1 = Solution().matrixReshape(m,134,38)

76 67 134 38
?


In [1144]:
len(m1)

134

In [1145]:
len(m1[0])

38

## 896. Monotonic Array




```
An array is monotonic if it is either monotone increasing or monotone decreasing.

An array A is monotone increasing if for all i <= j, A[i] <= A[j].  An array A is monotone decreasing if for all i <= j, A[i] >= A[j].

Return true if and only if the given array A is monotonic.
```

In [1149]:
class Solution:
    def isMonotonic(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        lens = len(A)
        flag = None
        m=0
        while(m < lens -1):
            if not flag:
                if A[m] == A[m+1]: pass
                else: flag = 1 if A[m] < A[m+1] else 2
            if flag:
                if A[m] < A[m+1]:
                    if flag == 2: return False
                elif A[m] > A[m+1]:
                    if flag ==1: return False
                else:
                    pass
            m+=1
            
        return True
               

1 line solution:...

In [1148]:
class Solution:
    def isMonotonic(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        return all(A[i]>=A[i+1] for i in range(len(A)-1)) or all(A[i]<=A[i+1] for i in range(len(A)-1))

## 888. Fair Candy Swap

```
Alice and Bob have candy bars of different sizes: A[i] is the size of the i-th bar of candy that Alice has, and B[j] is the size of the j-th bar of candy that Bob has.

Since they are friends, they would like to exchange one candy bar each so that after the exchange, they both have the same total amount of candy.  (The total amount of candy a person has is the sum of the sizes of candy bars they have.)

Return an integer array ans where ans[0] is the size of the candy bar that Alice must exchange, and ans[1] is the size of the candy bar that Bob must exchange.

If there are multiple answers, you may return any one of them.  It is guaranteed an answer exists.

 

Example 1:

Input: A = [1,1], B = [2,2]
Output: [1,2]
Example 2:

Input: A = [1,2], B = [2,3]
Output: [1,2]
Example 3:

Input: A = [2], B = [1,3]
Output: [2,3]
Example 4:

Input: A = [1,2,5], B = [2,4]
Output: [5,4]
```

In [1151]:
a=[1,2,3]

In [1154]:
sum(a[:0:]+a[::] )

5

In [1155]:
a[2::]

[3]

In [1156]:
class Solution:
    def fairCandySwap(self, A, B):
        """
        :type A: List[int]
        :type B: List[int]
        :rtype: List[int]
        """
        # A[:i]+A[i+1:]+B[j] == B[:j]+B[j+1:]+A[i]
        
        for i in range(0,len(A)):
            for j in range(0,len(B)):
                if B[j] + sum(A[:i] + A[i+1:]) == A[i] + sum(B[:j] + B[j+1:]):
                    return [A[i],B[j]]

In [1160]:
Solution().fairCandySwap([1,3,1],[2,2,3])

[1, 2]

In [1161]:
class Solution:
    def fairCandySwap(self, A, B):
        """
        :type A: List[int]
        :type B: List[int]
        :rtype: List[int]
        """
        # sumA -Ai + Bj = sumB - Bj + Ai
        # Bj - Ai = (sumB- sumA) / 2
        # Bj = (sumB-sumA)/2 + Ai?
        sumA = sum(A)
        sumB = sum(B)
        for i in range(0,len(A)):
            for j in range(0,len(B)):
                #same symbol
                if (sumA < sumB and A[i] > B[j]) or (sumA > sumB and A[i] < B[j]) : continue
                    
                if (sumA - A[i] + B[j] )== (sumB - B[j] + A[i]):
                    return [A[i],B[j]]

In [1162]:
Solution().fairCandySwap([1,1],[2,2])

[1, 2]

 超时...

大神门,很喜欢用set来加速循环的查找啊? 
```
前提是可以用set, 也要看情况的!  比如本题中,答案并不唯一, 要交换A中的1 ,并不要关心位置,而是它的值, 就可以用set
```

In [ ]:
class Solution:
    def fairCandySwap(self, A, B):
        dif = (sum(A) - sum(B)) / 2
        A = set(A)
        for b in set(B):
            if dif + b in A:
                return [dif + b, b]


## 485. Max Consecutive Ones



```
Given a binary array, find the maximum number of consecutive 1s in this array.

Example 1:
Input: [1,1,0,1,1,1]
Output: 3
Explanation: The first two digits or the last three digits are consecutive 1s.
    The maximum number of consecutive 1s is 3.
```

In [1179]:
class Solution:
    def findMaxConsecutiveOnes(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        maxs = 0
        n = 0
        lens = 0
        while n < len(nums):
            if nums[n] == 0:
                maxs = lens if lens > maxs else maxs
                lens = 0
            else:
                lens += 1
            n += 1
        maxs = lens if lens > maxs else maxs
        return maxs
        

In [1180]:
Solution().findMaxConsecutiveOnes([1,0,1,1,0,1])

2

## 283. Move Zeroes

```
Given an array nums, write a function to move all 0's to the end of it while maintaining the relative order of the non-zero elements.

Example:

Input: [0,1,0,3,12]
Output: [1,3,12,0,0]
```

In [1228]:
a=[1,2,3,0,0]
x = a.remove(0)
x = a.remove(0)
x = a.remove(0)
print(x)

ValueError: list.remove(x): x not in list

In [1231]:
class Solution:
    def moveZeroes(self, nums):
        """
        :type nums: List[int]
        :rtype: void Do not return anything, modify nums in-place instead.
        """
        zlen = 0
        while True:
            try:
                nums.remove(0)
                zlen += 1
            except:
                break
        while zlen >0:
            nums.append(0)
            zlen -= 1
        

In [1233]:
nums = [0,1,0,3,12]
Solution().moveZeroes(nums)
nums

[1, 3, 12, 0, 0]

In [1234]:
~1

-2

In [ ]:
class Solution:
    def moveZeroes(self, nums):
        for x in range(nums.count(0)):
            nums.remove(0)
            nums.append(0)

## 448. Find All Numbers Disappeared in an Array



```
Given an array of integers where 1 ≤ a[i] ≤ n (n = size of array), some elements appear twice and others appear once.

Find all the elements of [1, n] inclusive that do not appear in this array.

Could you do it without extra space and in O(n) runtime? You may assume the returned list does not count as extra space.

Example:

Input:
[4,3,2,7,8,2,3,1]

Output:
[5,6]
```

最简单的用set的场景了把...

In [ ]:
class Solution:
    def findDisappearedNumbers(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        s =set(nums)
        l = []
        for i in range(1, len(nums)+1):
            if i not in s:
                l.append(i)
        return l
        

## 169. Majority Element

In [ ]:
还是用dict之类的把 统计个数...前面看到的ConterDict完美

In [1238]:
import math

In [1239]:
math.ceil(5/2)

3

In [ ]:
from collections import Counter

In [1243]:
class Solution:
    def majorityElement(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        from collections import Counter
        d = Counter(nums)
        f = math.ceil(len(nums)/2)
        for i in d:
            if d[i] >= f:
                return i
        

In [1244]:
Solution().majorityElement([2,2,1,1,1,2,2])

2



```
Given an array of size n, find the majority element. The majority element is the element that appears more than ⌊ n/2 ⌋ times.

You may assume that the array is non-empty and the majority element always exist in the array.

Example 1:

Input: [3,2,3]
Output: 3
Example 2:

Input: [2,2,1,1,1,2,2]
Output: 2
```

## 121. Best Time to Buy and Sell Stock

```
Say you have an array for which the ith element is the price of a given stock on day i.

If you were only permitted to complete at most one transaction (i.e., buy one and sell one share of the stock), design an algorithm to find the maximum profit.

Note that you cannot sell a stock before you buy one.

Example 1:

Input: [7,1,5,3,6,4]
Output: 5
Explanation: Buy on day 2 (price = 1) and sell on day 5 (price = 6), profit = 6-1 = 5.
             Not 7-1 = 6, as selling price needs to be larger than buying price.
Example 2:

Input: [7,6,4,3,1]
Output: 0
Explanation: In this case, no transaction is done, i.e. max profit = 0.

Input: [7,2,5,3,6,1,2]
Output: 5

```

In [ ]:
```
每遇到低点,都以后面的最高点卖出, 每次得到price, 遍历所有的低点,得到最终的max
```

In [1578]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n = 0 
        # 0 sold out 
        # 1 bought in
        status = 0
        start = end = 0
        profit = 0
        high_price = 0
        while n < len(prices) - 1:
            
            if status == 0:
                if prices[n] >= prices[n+1]:
                    pass
                else:
                    status = 1
                    start = n
            if status == 1:
                high_price = max(prices[n:])
                profit = max(profit, high_price - prices[start])
                status = 0 
            n+=1
        if status == 1:  profit = max (profit, (prices[-1] - prices[start]) )
        return profit
        

In [1579]:
Solution().maxProfit([7,1,5,3,6,4])

5

In [1580]:
 Solution().maxProfit([7,2,5,3,6,1,2])

4

##  122. Best Time to Buy and Sell Stock II




```
Say you have an array for which the ith element is the price of a given stock on day i.

Design an algorithm to find the maximum profit. You may complete as many transactions as you like (i.e., buy one and sell one share of the stock multiple times).

Note: You may not engage in multiple transactions at the same time (i.e., you must sell the stock before you buy again).

Example 1:

Input: [7,1,5,3,6,4]
Output: 7
Explanation: Buy on day 2 (price = 1) and sell on day 3 (price = 5), profit = 5-1 = 4.
             Then buy on day 4 (price = 3) and sell on day 5 (price = 6), profit = 6-3 = 3.
Example 2:

Input: [1,2,3,4,5]
Output: 4
Explanation: Buy on day 1 (price = 1) and sell on day 5 (price = 5), profit = 5-1 = 4.
             Note that you cannot buy on day 1, buy on day 2 and sell them later, as you are
             engaging multiple transactions at the same time. You must sell before buying again.
Example 3:

Input: [7,6,4,3,1]
Output: 0
Explanation: In this case, no transaction is done, i.e. max profit = 0.
```

```
[7,2,1,5,3,6,4]
status  = sold_out
buy first  要不要买? 判断趋势 cur >= next , no  cur <= next? yes
start = xxx
status = bought

要不要卖? cur >= next ? yes 1 cur < next ? no ! 
```

没啥巧的 理解题意,用status的状态机的方式, 没有用啥数据结构

In [1569]:
class Solution:
    def maxProfit(self, prices):
        """
        :type prices: List[int]
        :rtype: int
        """
        n = 0 
        # 0 sold out 
        # 1 bought in
        status = 0
        start = end = 0
        profit = 0
        while n < len(prices) - 1:
            
            if status == 0:
                if prices[n] >= prices[n+1]:
                    pass
                else:
                    status = 1
                    start = n
            if status == 1:
                if prices[n] > prices[n+1]:
                    status =0
                    end = n
                    profit += (prices[end] - prices[start])
                    start = end = 0
                else:
                    pass          
            
            n+=1
        if status == 1:  profit += (prices[-1] - prices[start])
        return profit
        

In [1570]:
Solution().maxProfit([7,1,5,3,6,4])

7

In [1253]:
Solution().maxProfit([1,2,3,4,5])

4

In [1254]:
Solution().maxProfit([5,4,2,1,1])

0

In [1255]:
Solution().maxProfit([5,4,2,1,1,2])

1

In [1256]:
Solution().maxProfit([1,4,3,7,6,8])

9

大神说: 所有上升的都是profit! 牛逼....仔细想想,应该没上面的快,

In [1258]:
class Solution:
    def maxProfit(self, prices):
        return sum([ prices[i+1]-prices[i] for i in range(len(prices)-1) if prices[i+1]-prices[i]>0])

## 217. Contains Duplicate

```
Given an array of integers, find if the array contains any duplicates.

Your function should return true if any value appears at least twice in the array, and it should return false if every element is distinct.

Example 1:

Input: [1,2,3,1]
Output: true
Example 2:

Input: [1,2,3,4]
Output: false
Example 3:

Input: [1,1,1,3,3,4,3,2,4,2]
Output: true
```

In [1260]:
set([1,2,3])

{1, 2, 3}

In [ ]:
class Solution:
    def containsDuplicate(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        return len(set(nums)) < len(nums)

## 717. 1-bit and 2-bit Characters



```
We have two special characters. The first character can be represented by one bit 0. The second character can be represented by two bits (10 or 11).

Now given a string represented by several bits. Return whether the last character must be a one-bit character or not. The given string will always end with a zero.

Example 1:
Input: 
bits = [1, 0, 0]
Output: True
Explanation: 
The only way to decode it is two-bit character and one-bit character. So the last character is one-bit character.
Example 2:
Input: 
bits = [1, 1, 1, 0]
Output: False

bits = [1, 0, 1, 1, 0]
Output: True

bits = [0, 1, 0, 1, 1, 0]
Output: True
Explanation: 
The only way to decode it is two-bit character and two-bit character. So the last character is NOT one-bit character.
```

```
从头扫描到n-1:
    if 0:  忽略
        n+=1
    if 1: 
        n+=2
最后看n到哪了: if n == lens - 1 , 前面刚好成对,最后1个0 必然是single char ,其余情况则是配对的, 最后1个必然是1,0
```

In [1280]:
class Solution:
    def isOneBitCharacter(self, bits):
        """
        :type bits: List[int]
        :rtype: bool
        """
        lens = len(bits)
        if bits[lens-2] is 0: return True
        n  = 0
        while n < (lens -1):
            if bits[n] == 0:
                n+=1
            if bits[n] == 1:
                n+=2
        print(n)
        if n == (lens - 1): return True
        else : return False
                
        

In [1281]:
Solution().isOneBitCharacter([1, 0, 0])

True

In [1282]:
Solution().isOneBitCharacter([1, 1, 1, 0])

4


False

In [1283]:
Solution().isOneBitCharacter([1,0, 1, 1, 0])

4


True

In [1284]:
Solution().isOneBitCharacter([0,1,0, 1, 1, 0])

5


True

In [1285]:
Solution().isOneBitCharacter([0,0,0, 0, 1, 0])

6


False

In [1286]:
Solution().isOneBitCharacter([0, 1, 0])

3


False

## 167. Two Sum II - Input array is sorted

```
Given an array of integers that is already sorted in ascending order, find two numbers such that they add up to a specific target number.

The function twoSum should return indices of the two numbers such that they add up to the target, where index1 must be less than index2.

Note:

Your returned answers (both index1 and index2) are not zero-based.
You may assume that each input would have exactly one solution and you may not use the same element twice.
Example:

Input: numbers = [2,7,11,15], target = 9
Output: [1,2]
Explanation: The sum of 2 and 7 is 9. Therefore index1 = 1, index2 = 2.
```

In [1290]:
def binary_search(start, end, nums,val):
    mid = (start + end)//2 
    if nums[mid] == val: return mid
    if start >= end: return None
    if nums[mid] > val : return binary_search(start,mid-1,nums,val)
    if nums[mid] < val : return binary_search(mid+1,end,nums,val)
    
class Solution:
    def twoSum(self, numbers, target):
        """
        :type numbers: List[int]
        :type target: int
        :rtype: List[int]
        """
        for idx,i in enumerate(numbers):
            ret =  binary_search(idx+1,len(numbers)-1,numbers, target-i)
            if ret:
                return [ idx+1, ret+1 ] if i <= numbers[ret] else [ ret+1,idx+1]
        

改了几次: 
```
index 从1开始;
binary_search 不需要从0开始,而是从idx+1开始,即不搜索i 自己.

```

不够快啊,想用set? 但是返回值是index,

想到的是hash表,比Counter还要猛一点,
```
0 ---> 5 ---> 6
1----> 7 ----8
2----> 2 ---> 3 ---4
```
遍历 表的键值, 如果存在 0 + 1 =1 , return 对应桶的首个index
如果不存在 ,有一种情况是 1 + 1 =2, 即两个元素位于同一个桶,


In [1316]:
a={"1":[],"2":2}

In [1317]:
for  k,v in a.items():
    print(k,v)
    

1 []
2 2


普通的dict, 键值相同时,后者覆盖了前者, 如果不覆盖, 其键值应该是一个generator, 可能1个key可能有多个值.

In [1356]:
class HashTable:
    def __init__(self,l):
        self.keys = set(l)
        self.dicts = {k:[] for k in self.keys }
        for i,v in enumerate(l):
            self.dicts[v].append(i)
    def __getitem__(self,key):
        return self.dicts[key] if key in self.keys else None
    def __str__(self):
        return  "{\n%s\n}"% (["%s:%s"%(k,str(v)) for k,v in self.dicts.items()])

In [1375]:
a = HashTable([3,3,2,2,2,0,0,1,1] )

In [1376]:
print(a)

{
['0:[5, 6]', '1:[7, 8]', '2:[2, 3, 4]', '3:[0, 1]']
}


In [1373]:
class HashTable:
    def __init__(self,l):
        self.keys = set(l)
        self.dicts = {k:[] for k in self.keys }
        for i,v in enumerate(l):
            self.dicts[v].append(i)
    def __getitem__(self,key):
        return self.dicts[key] if key in self.keys else None
    def __str__(self):
        return  "{\n%s\n}"% (["%s:%s"%(k,str(v)) for k,v in self.dicts.items()])
    
class Solution:
    def twoSum(self, numbers, target):
        """
        :type numbers: List[int]
        :type target: int
        :rtype: List[int]
        """
        ht = HashTable(numbers)
        for i in ht.keys:
            if target-i in ht.keys:
                if (target -i) == i :
                    if len(ht[i]) > 1:
                        return [ht[i][0] + 1, ht[i][1] + 1]
                    else: pass
                else:
                    return [ht[i][0] + 1 ,ht[target-i][0] + 1] if i <=  target -i else  \
                    [ht[target-i][0] + 1,ht[i][0] + 1]
                
            

        
        

In [1374]:
Solution().twoSum([2,7,11,15],9)

1


[1, 2]

In [1372]:
Solution().twoSum([0,1,1,2,3,4,15],5)

[2, 6]

更快的方法, 遍历元素,先存到dict中, 再遇到元素 看是否和dict中满足条件,满足结束 over

In [1377]:
class Solution:
    def twoSum(self, numbers, target):
        dic = {}
        for i, num in enumerate(numbers):
            if target-num in dic:
                return [dic[target-num]+1, i+1]
            dic[num] = i

## 697. Degree of an Array

```
Given a non-empty array of non-negative integers nums, the degree of this array is defined as the maximum frequency of any one of its elements.

Your task is to find the smallest possible length of a (contiguous) subarray of nums, that has the same degree as nums.

Example 1:
Input: [1, 2, 2, 3, 1]
Output: 2
Explanation: 
The input array has a degree of 2 because both elements 1 and 2 appear twice.
Of the subarrays that have the same degree:
[1, 2, 2, 3, 1], [1, 2, 2, 3], [2, 2, 3, 1], [1, 2, 2], [2, 2, 3], [2, 2]
The shortest length is 2. So return 2.
Example 2:
Input: [1,2,2,3,1,4,2]
Output: 6
```

## 661. Image Smoother

```
Given a 2D integer matrix M representing the gray scale of an image, you need to design a smoother to make the gray scale of each cell becomes the average gray scale (rounding down) of all the 8 surrounding cells and itself. If a cell has less than 8 surrounding cells, then use as many as you can.

Example 1:
Input:
[[1,1,1],
 [1,0,1],
 [1,1,1]]
Output:
[[0, 0, 0],
 [0, 0, 0],
 [0, 0, 0]]
Explanation:
For the point (0,0), (0,2), (2,0), (2,2): floor(3/4) = floor(0.75) = 0
For the point (0,1), (1,0), (1,2), (2,1): floor(5/6) = floor(0.83333333) = 0
For the point (1,1): floor(8/9) = floor(0.88888889) = 0
Note:
The value in the given matrix is in the range of [0, 255].
The length and width of the given matrix are in the range of [1, 150].

```

周边加一圈0 , padding的思想, so easy..

In [1468]:
M=[[1]]

In [1469]:
def find_neighbours(row,col,i,j):
    counts = 9
    if i+1 >= row: counts -=3
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1469-5f1faa98f909>, line 4)

In [1466]:
find_neighbours(M,0,0)

4

In [1520]:
a=[1]

In [1521]:
a.count(1)

1

```
思路就是padding, 但是tricky的是用-1来padding ,完美防止与0的冲突..
```

In [1535]:
from math import floor
class Solution:
    def imageSmoother(self, M):
        """
        :type M: List[List[int]]
        :rtype: List[List[int]]
        """
        import copy 
        ret = copy.deepcopy(M)
        row = len(M)
        col = len(M[0])
        new_row,new_col = row+2,col+2
        flag = -1
        z_row = [-1 for i in range(0,new_col)]
        for i in M:
            i.insert(0,-1)
            i.append(-1)
        M.insert(0,z_row)
        M.append(z_row)
#         print(M)
        for i in range(1,new_row -1):
            for j in range(1,new_col-1):
                
                ceils = [ M[i][j] , M[i+1][j] , M[i-1][j] ,
                         M[i][j+1] , M[i][j-1] , M[i+1][j+1] ,
                         M[i-1][j-1] , M[i+1][j-1] , M[i-1][j+1] ]
                miss = ceils.count(flag)
                # ave is different !
#                 print(9-miss)
                ret[i-1][j-1] = floor ( (sum(ceils) + miss) / (9-miss)) 
        return ret

In [1532]:
Solution().imageSmoother([[1,1,1],[1,0,1],[1,1,1]])

[[-1, -1, -1, -1, -1], [-1, 1, 1, 1, -1], [-1, 1, 0, 1, -1], [-1, 1, 1, 1, -1], [-1, -1, -1, -1, -1]]
4
6
4
6
9
6
4
6
4


[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

In [1533]:
Solution().imageSmoother([[1]])

[[-1, -1, -1], [-1, 1, -1], [-1, -1, -1]]
1


[[1]]

In [1534]:
Solution().imageSmoother([[2,3]])

[[-1, -1, -1, -1], [-1, 2, 3, -1], [-1, -1, -1, -1]]
2
2


[[2, 2]]

In [1531]:
Solution().imageSmoother([[2,3,4],[5,6,7],[8,9,10],[11,12,13],[14,15,16]])

[[-1, -1, -1, -1, -1], [-1, 2, 3, 4, -1], [-1, 5, 6, 7, -1], [-1, 8, 9, 10, -1], [-1, 11, 12, 13, -1], [-1, 14, 15, 16, -1], [-1, -1, -1, -1, -1]]
4
6
4
6
9
6
6
9
6
6
9
6
4
6
4


[[4, 4, 5], [5, 6, 6], [8, 9, 9], [11, 12, 12], [13, 13, 14]]

## 268. Missing Number

```
Given an array containing n distinct numbers taken from 0, 1, 2, ..., n, find the one that is missing from the array.

Example 1:

Input: [3,0,1]
Output: 2
Example 2:

Input: [9,6,4,2,3,5,7,0,1]
Output: 8
```

In [1536]:
class Solution:
    def missingNumber(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        sets = set(nums)
        for i in range(0,len(nums)+1):
            if i not in sets:
                return i

## 830. Positions of Large Groups

```
In a string S of lowercase letters, these letters form consecutive groups of the same character.

For example, a string like S = "abbxxxxzyy" has the groups "a", "bb", "xxxx", "z" and "yy".

Call a group large if it has 3 or more characters.  We would like the starting and ending positions of every large group.

The final answer should be in lexicographic order.

 

Example 1:

Input: "abbxxxxzzy"
Output: [[3,6]]
Explanation: "xxxx" is the single large group with starting  3 and ending positions 6.
Example 2:

Input: "abc"
Output: []
Explanation: We have "a","b" and "c" but no large group.
Example 3:

Input: "abcdddeeeeaabbbcd"
Output: [[3,5],[6,9],[12,14]]
```

In [ ]:
前面造的HashTable很适合呀!


In [1567]:
class Solution:
    def largeGroupPositions(self, S):
        """
        :type S: str
        :rtype: List[List[int]]
        """
        ret = []
        last = -1
        start = 0
        for i,v in enumerate(S):
            if last != v:
                if (i - start) >=3: ret.append([start,i-1])
                start = i
            last = v
        if (i-start) >=2: ret.append([start,i])
        return ret
            
            

In [1566]:
Solution().largeGroupPositions("abcdddeeeeaabbbcd")

[[3, 5], [6, 9], [12, 14]]

## 628. Maximum Product of Three Numbers

```
Given an integer array, find three numbers whose product is maximum and output the maximum product.

Example 1:
Input: [1,2,3]
Output: 6
Example 2:
Input: [1,2,3,4]
Output: 24
Note:
The length of the given array will be in range [3,104] and all elements are in the range [-1000, 1000].
Multiplication of any three numbers in the input won't exceed the range of 32-bit signed integer.
```

找3个数的乘积最大. CounterDict...找到最大的, 然后看个数 ,>=3 <3 在看第2个.

没注意负数!,考虑负数就没那么简单了.

In [1589]:
from collections import Counter

In [1590]:
d = Counter()

In [ ]:
d.pop

In [ ]:
全为正/全为负/1个负/2个以上负数

In [1612]:
any(i < 0 for i in [1,0,1])

False

In [1672]:
maxs =4
maxs if maxs <=3 else 3

3

In [1707]:
def cal_products(cd, count, way='pos'):
    maxs = max(cd.keys()) if way == 'pos' else min(cd.keys())
    counts = count
    products = 1
    while(counts > 0):
        while cd[maxs] >0 and counts > 0:
            products *= maxs 
            cd[maxs]-=1
            counts -=1
        if counts <=0: break
        else:
            cd.pop(maxs)
            maxs = max(cd.keys()) if way == 'pos' else min(cd.keys())
    return products


from collections import Counter
class Solution:
    def maximumProduct(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        positive = nagtive = maxs = mins = 0
        products = 1
        for i in nums:
            if i >=0: positive += 1
            if i < 0: nagtive += 1
        cd = Counter(nums)
        if nagtive == 0 or nagtive == 1 or positive == 0:
            return cal_products(cd,3,'pos')
        else:
            assert(nagtive >= 2)
            nag_max = cal_products(cd ,2, 'nag') *  max(cd.keys()) 
            if positive == 1: 
                return nag_max
            else:
                pos_max = cal_products(cd ,3, 'pos')
                return  max(pos_max, nag_max)
                           

In [1708]:
Solution().maximumProduct([-1,-2,-3,-4,-5])

-6

In [1698]:
Solution().maximumProduct([1,2,3,4,4])

48

In [1699]:
Solution().maximumProduct([1,-10,-2,2,3])

60

没考虑负数!

In [1700]:
Solution().maximumProduct([-4,-3,-2,-1,60])


720

In [1701]:
Solution().maximumProduct([-4,3,2,-4,60])

960

In [1702]:
16*60

960

In [1703]:
Solution().maximumProduct([-4,-4,-4,-4,4])

64

In [1704]:
Solution().maximumProduct([-1,-1,-4, 4, 4])

16

太勉强了....

看大神思路... 不分那么细, 反正结果总是max1 * max( (min1*min2) , (max2*max3)

In [1709]:
class Solution(object):
    def maximumProduct(self, nums):
        max1 = max2 = max3 = float("-inf")
        min1 = min2 = float("inf")        
        for n in nums:
            if n > max3:
                max1, max2, max3 = sorted([max1, max2, n], reverse=True)
            if n < min2:
                min1, min2 = sorted([min1, n])                    
        return max1*max(max2*max3, min1*min2)

## 746. Min Cost Climbing Stairs

```
On a staircase, the i-th step has some non-negative cost cost[i] assigned (0 indexed).

Once you pay the cost, you can either climb one or two steps. You need to find minimum cost to reach the top of the floor, and you can either start from the step with index 0, or the step with index 1.

Example 1:
Input: cost = [10, 15, 20]
Output: 15
Explanation: Cheapest is start on cost[1], pay that cost and go to the top.
Example 2:
Input: cost = [1, 100, 1, 1, 1, 100, 1, 1, 100, 1]
Output: 6
Explanation: Cheapest is start on cost[0], and only step on 1s, skipping cost[3].
Note:
cost will have a length in the range [2, 1000].
Every cost[i] will be an integer in the range [0, 999].
```

最优路径的问题?是不是要上算法...其实本质上即使贪婪算法
理解问题, 只有第1步骤可以选,后面不能选

In [1882]:
min(1,2,3)

1

In [1881]:
min([1,2,3])

1

In [1962]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """
        lens = len(cost)
        cost.append(0)
        cost.append(0)
        ret1 = ret2 = 0
        
        def ClimbingStairs(_cost,start,end):
            if start == end - 1:
                return min(_cost[start],_cost[start+1]) if start ==0 else _cost[start]
            if start == end: return _cost[start]
            else:
                a = (
                    ClimbingStairs(cost,start+1,end),
                    _cost[start]+ ClimbingStairs(cost,start+2,end),
                    _cost[start+1]+ ClimbingStairs(cost,start+2,end))
                return min(a) if start == 0 else  \
                    _cost[start] + \
                    min(ClimbingStairs(cost,start+1,end),ClimbingStairs(cost,start+2,end))
            
        return ClimbingStairs(cost,0,lens-1)
                
                

In [1963]:
Solution().minCostClimbingStairs ( [0,1,1,0] )

start=0 end=3
[0, 1, 1, 0, 0, 0]
start=1 end=3
[1, 1, 0, 0, 0]
start=2 end=3
[1, 0, 0, 0]
start=3 end=3
[0, 0, 0]
start=3 end=3
[0, 0, 0]
start=2 end=3
[1, 0, 0, 0]
start=3 end=3
[0, 0, 0]
start=2 end=3
[1, 0, 0, 0]
start=2 end=3
[1, 0, 0, 0]


1

In [1964]:
Solution().minCostClimbingStairs ( [10, 15, 20, 1] )

start=0 end=3
[10, 15, 20, 1, 0, 0]
start=1 end=3
[15, 20, 1, 0, 0]
start=2 end=3
[20, 1, 0, 0]
start=3 end=3
[1, 0, 0]
start=3 end=3
[1, 0, 0]
start=2 end=3
[20, 1, 0, 0]
start=3 end=3
[1, 0, 0]
start=2 end=3
[20, 1, 0, 0]
start=2 end=3
[20, 1, 0, 0]


16

In [ ]:
Solution().minCostClimbingStairs ([1, 100, 1, 1, 1, 100, 1, 1, 100, 1] )

In [1967]:
Solution().minCostClimbingStairs([0,1,2,2])

2

第1步就可以选! 不一定要从_cost[0] 开始

In [1952]:
Solution().minCostClimbingStairs([1,0,0,0])

start=0 end=3
[1, 0, 0, 0, 0, 0]
start=3 end=3
[0, 0, 0]
start=2 end=3
[0, 0, 0, 0]
start=2 end=3
[0, 0, 0, 0]
start=1 end=3
[0, 0, 0, 0, 0]
(0, 1, 0, 1)


0

严重的超时了, 要想想其他策略,动态规划的算法:

In [9]:
class Solution:
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """    
        prevPrev, prev  = cost[0], cost[1]
        for i in range(2, len(cost)):
            cur = min(prevPrev, prev) + cost[i]
            prevPrev, prev = prev, cur
        return min(prevPrev, prev)


子问题是: 到第n步的最小代价为dp[n]
```   
第1步: cost[0]
第2步: cost[1] 
第n步: cost[n] + min(dp[n-1],dp[n-2])
```
    
因为有2种可能的start, 因此需要同时更新2个  

In [10]:
Solution().minCostClimbingStairs([841,462,566,398,243,248,238,650,989,576,361,126,334,729,446,897,953,38,195,679,65,707,196,705,569,275,259,872,630,965,978,109,56,523,851,887,91,544,598,963,305,481,959,560,454,883,50,216,732,572,511,156,177,831,122,667,548,978,771,880,922,777,990,498,525,317,469,151,874,202,519,139,670,341,514,469,858,913,94,849,839,813,664,163,3,802,21,634,944,901,446,186,843,742,330,610,932,614,625,169,833,4,81,55,124,294,71,24,929,534,621,543,417,534,427,327,179,90,341,949,368,692,646,290,488,145,273,617,596,82,538,751,80,616,763,826,932,184,630,478,163,925,259,237,839,602,60,786,603,413,816,278,4,35,243,64,631,405,23,638,618,829,481,877,756,482,999,973,718,157,262,752,931,882,741,40,77,535,542,879,607,879,321,46,210,116,244,830,591,285,382,925,48,497,913,203,239,696,162,623,291,525,950,27,546,293,108,577,672,354,256,3,671,998,22,989,557,424,251,923,542,243,46,488,80,374,372,334,190,817,150,742,362,196,75,193,162,645,859,758,433,903,199,289,175,303,475,818,213,576,181,668,243,297,572,549,840,161,292,719,226,338,981,345,203,655,210,65,111,746,76,935,406,646,976,567,32,726,638,674,727,861,426,297,349,464,973,341,452,826,223,805,940,458,468,967,107,345,987,553,407,916,103,324,367,864,74,946,712,596,105,194,79,634,855,703,70,170,543,208,739,632,663,880,857,824,258,743,488,659,647,470,958,492,211,927,356,488,744,570,143,674,502,589,270,80,6,463,506,556,495,713,407,229,689,280,162,454,757,565,267,575,417,948,607,269,852,938,560,24,222,580,604,800,628,487,485,615,796,384,555,226,412,445,503,810,949,966,28,768,83,213,883,963,831,390,951,378,497,440,780,209,734,290,96,398,146,56,445,880,910,858,671,164,552,686,748,738,837,556,710,787,343,137,298,685,909,828,499,816,538,604,652,7,272,729,529,343,443,593,992,434,588,936,261,873,64,177,827,172,712,628,609,328,672,376,628,441,9,92,525,222,654,699,134,506,934,178,270,770,994,158,653,199,833,802,553,399,366,818,523,447,420,957,669,267,118,535,971,180,469,768,184,321,712,167,867,12,660,283,813,498,192,740,696,421,504,795,894,724,562,234,110,88,100,408,104,864,473,59,474,922,759,720,69,490,540,962,461,324,453,91,173,870,470,292,394,771,161,777,287,560,532,339,301,90,411,387,59,67,828,775,882,677,9,393,128,910,630,396,77,321,642,568,817,222,902,680,596,359,639,189,436,648,825,46,699,967,202,954,680,251,455,420,599,20,894,224,47,266,644,943,808,653,563,351,709,116,849,38,870,852,333,829,306,881,203,660,266,540,510,748,840,821,199,250,253,279,672,472,707,921,582,713,900,137,70,912,51,250,188,967,14,608,30,541,424,813,343,297,346,27,774,549,931,141,81,120,342,288,332,967,768,178,230,378,800,408,272,596,560,942,612,910,743,461,425,878,254,929,780,641,657,279,160,184,585,651,204,353,454,536,185,550,428,125,889,436,906,99,942,355,666,746,964,936,661,515,978,492,836,468,867,422,879,92,438,802,276,805,832,649,572,638,43,971,974,804,66,100,792,878,469,585,254,630,309,172,361,906,628,219,534,617,95,190,541,93,477,933,328,984,117,678,746,296,232,240,532,643,901,982,342,918,884,62,68,835,173,493,252,382,862,672,803,803,873,24,431,580,257,457,519,388,218,970,691,287,486,274,942,184,817,405,575,369,591,713,158,264,826,870,561,450,419,606,925,710,758,151,533,405,946,285,86,346,685,153,834,625,745,925,281,805,99,891,122,102,874,491,64,277,277,840,657,443,492,880,925,65,880,393,504,736,340,64,330,318,703,949,950,887,956,39,595,764,176,371,215,601,435,249,86,761,793,201,54,189,451,179,849,760,689,539,453,450,404,852,709,313,529,666,545,399,808,290,848,129,352,846,2,266,777,286,22,898,81,299,786,949,435,434,695,298,402,532,177,399,458,528,672,882,90,547,690,935,424,516,390,346,702,781,644,794,420,116,24,919,467,543,58,938,217,502,169,457,723,122,158,188,109,868,311,708,8,893,853,376,359,223,654,895,877,709,940,195,323,64,51,807,510,170,508,155,724,784,603,67,316,217,148,972,19,658,5,762,618,744,534,956,703,434,302,541,997,214,429,961,648,774,244,684,218,49,729,990,521,948,317,847,76,566,415,874,399,613,816,613,467,191])

209040

### 插播DP的理解:

例子:
```
最长上升子序列LIS:

输入n及一个长度为n的数列，求出此序列的最长上升子序列长度。
上升子序列指的是对于任意的i<j都满足ai<aj的子序列。（1<=n<=1000,0<=ai<=1000000）
```

三个原则实施DP:
```
1. 可划分阶段
2. 最优化原理,即子问题的局部最优将导致整个问题的全局最优，即问题具有最优子结构的性质,
   也就是说一个问题的最优解只取决于其子问题的最优解
3. 无后效性,即当前的状态只和过去有关,和未来无关,有点像线性时不变里的因果性
```

#### 划分1


```
最长上升子序列可能以任何位置作为结尾! 这就是`划分子问题`(方式并不唯一!), 即: dp[n]是以a[n]结尾的最长子序列的长度;

dp[n]可以递推, 其值可能和dp[0],dp[1],...dp[n-1]有关, 这是`无后效性`

所有的dp[n]求得时,全局问题自然得解,即max(dp)
```

dp的值要么为1,要么为其他,因此初始值全部为1即可



```
step: 0 , dp[0] = 1
step: i, 对于j in range(1,i) a[j]是dp[j]代表的最长子序列的结尾, if a[i] > a[j],此时可以形成新的最长子序列 长度为a[j]+1,
    找到所有dp[j]中,可能的最大的dp[j]+1, 用来更新dp[i]
```

In [76]:
class Solution:
    def maxSubArrayLength(self,nums):
        dp = []
        [dp.append(1) for i in range(0,len(nums))]
        print(dp)
        for i in range(1,len(nums)):
            for j in range(1,i):
                dp[i] = max(dp[i],dp[j]+1) if nums[i] > nums[j] else dp[i]
        print(dp)
        return max(dp)


In [77]:
Solution().maxSubArrayLength([4,2,3,1,100,2,3,4,5,6,7,6,6,8])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 2, 1, 3, 2, 3, 4, 5, 6, 7, 6, 6, 8]


8

#### 划分2

再来一种划分子问题的思路: 
```
dp[i]: 长度为i+1的上升子序列中末尾元素的最小值（不存在的话就是INF）。 i是递归的变量 最大长度i+1 == len(nums)

不存在,则标记为初值INF

100 99 91 92 93 94 95 1 2 3 4 5 3 
100 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
99 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 94 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
91 92 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 92 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 93 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 94 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 95 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 5 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
1 2 3 4 5 99999999 99999999 99999999 99999999 99999999 99999999 99999999 99999999 
5

```

In [22]:
class Solution:
    def maxSubArrayLength(self,nums):
        import sys
        dp = []
        INTMAX =999
        [dp.append(INTMAX) for i in range(0,len(nums))]
#         print(dp)
        
        dp[0] = nums[0]
        for j in range(1,len(nums)):
            for i in range(0,len(nums)):
                if nums[j] < dp[i]: 
                    dp[i] = min(nums[j],dp[i])
                    break
            print(dp)
            
        return dp.index(999)


In [25]:
Solution().maxSubArrayLength([100, 99 ,91, 92 ,93,94 , 1, 2 ,3 ,4, 5, 3 ])

[99, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 93, 999, 999, 999, 999, 999, 999, 999, 999, 999]
[91, 92, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 92, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 93, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 94, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 4, 999, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 4, 5, 999, 999, 999, 999, 999, 999, 999]
[1, 2, 3, 3, 5, 999, 999, 999, 999, 999, 999, 999]


5

求两个字符串的公共子序列的最大长度. 公共子串是相同的子串.
```
dp[i-1][j-1]为a为长度i,b为长度j时的公共子串的长度
dp[i]d[j] = dp[i-1][j-1] + 1 if a[i-1]==b[j-1]
否则 dp[i]d[j]  = max(dp[i]d[j-1] ,dp[i]d[j-1] )
```

In [45]:
class Solution:
    def maxSubArrayLength(self,a,b):
        lena = len(a)
        lenb = len(b)
        dp = []
        for i in range(0,lena):
            dp.append([])
            for j in range(0,lenb):
                dp[i].append(0)
        print(dp)
        dp[0][0] = 1 if a[0] == b[0] else 0
        for i in range(0,lena-1):
            for j in range(0,lenb-1):
                dp[i+1][j+1] = dp[i][j] + 1 if a[i] == b[j] else max(dp[i+1][j],dp[i][j+1])
        print(dp)
        return  dp[lena-1][lenb-1]       
                


In [46]:
Solution().maxSubArrayLength([2,3],[2,3,4])

[[0, 0, 0], [0, 0, 0]]
[[1, 0, 0], [0, 2, 2]]


2